<a href="https://colab.research.google.com/github/CalebRHaynes/social_doors_college/blob/master/soc_doors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Neuroimaging Data and Behavioral Files



In [119]:
import glob 
import pandas as pd
!ls

drive  sample_data


In [182]:
class Subject:
  
  def __init__(self, subnum):
    self.subnum = subnum

  def path_get(self, path):
  
    root_dir = 'drive/My Drive/social_doors_college/'

    mriqc_data = root_dir + f'mriqc/reports/sub-{self.subnum}_task*'
    #functional_data = 
    #T1 =
    
    beh_timing_files = glob.glob(root_dir + f'/behavior/EVfiles/{self.subnum}/*/*')
    
    #filetype- excel
    raw_behavioral = glob.glob(f'drive/My Drive/social_doors_college/behavior/raw/s{self.subnum}/*/*.xlsx')
    
    if path == 'L1':
      req = glob.glob(f'/content/drive/My Drive/social_doors_college/lowerLv_results/sub-{self.subnum}/L1*.feat')
    elif path == 'L2':
      req = glob.glob(f'/content/drive/My Drive/social_doors_college/lowerLv_results/sub-{self.subnum}/L2*.gfeat')
    return req

  def get_l1_files(self):
    return self.path_get('L1')
  
  def get_l2_files(self):
    return self.path_get('L2')

  def get_rvst(self):
    df = pd.read_csv('/content/drive/My Drive/social_doors_college/code/nm_rvst.csv')
    print(df.loc[df['subno'] == self.subnum])

p1 = Subject(3836).get_rvst()



   subno        RVST
0   3836  116.035119


In [166]:
class Dataset:
  def __init__(self, sublist):
    self.sublist = sublist
    pass

  #fmriprepdir 
  
  
  #sub_list = '/content/drive/My Drive/social_doors_college/code/sub_list.txt'
  
  def get_sublist(self):
    with open(self.sublist, "r") as file:
      lines = file.readlines()
    
    subject_list = []
      
    for s in lines:
      as_list = s.split(", ")
      subject_list.append(as_list[0].replace("\n", ""))

    file.close()
      
    return subject_list

  #masks
  

  def make_data_frame(self):
    pass



full_sub_list = Dataset('/content/drive/My Drive/social_doors_college/code/sub_list.txt').get_sublist()
print(full_sub_list)

['Subjects', 'sub-3836', 'sub-3845', 'sub-3846', 'sub-3847', 'sub-3849', 'sub-3851', 'sub-3852', 'sub-3855', 'sub-3864', 'sub-3865', 'sub-3871', 'sub-3877', 'sub-3883', 'sub-3886', 'sub-3887', 'sub-3889', 'sub-3890', 'sub-3891', 'sub-3892', 'sub-3893', 'sub-3895', 'sub-3910', 'sub-3912', 'sub-3920', 'sub-3967', 'sub-3992', 'sub-4017', 'sub-4018', 'sub-4019', 'sub-4020']


# Onset Extraction, EV File Generate

In [83]:
import pandas as pd
import numpy as np
import re
from collections import defaultdict
import csv
import json
!pwd

/content


In [84]:
#EV File (3 col) Generation
def event_writer(event_type, df_series, duration, block_type):
    '''
    Funciton takes a string, pandas series, and a float/int
    This function 
    1. Converts ms in series to s, and makes dataframe obj
    2. Duration is appended as a 'default value' column
    3. Makes column of ones (expected by fsl) and adds to df
    4. Writes out to csv with index and header turned off
    '''
    df = (df_series/1000).to_frame()
    df['duration'] = duration
    df['mag'] = 1.0
    df.to_csv(f'../behavior/EVfiles/{sub}/{condition_type}/run-0{run}_{block_type}_{event_type}.txt', index = False, header = False, sep = '\t')
    

def extract_fd_vals(json_file):
    f = open(json_file)
    data = json.load(f)
    fd_data = data['fd_mean'] 
    f.close() 
    return fd_data

In [85]:
#make a list for logging an intermediate file

interm = {}

'''
Split excel files into 4, convert to 3 'colmumn file' within sub folders

iterate over each excel file - 2 per person: social, nonsocial

2 blocks within each run

4 EVs in question: instruction, correct, incorect and decision

'''

for excel in glob.glob('../behavior/raw/s*/*/*.xlsx')[:]:
    
    #get sub number from filename
    sub = excel[17:21]
    print(sub)
    #load into pandas dataframe 
    df = pd.read_excel(excel, header = 1)
    

    #define block list
    split_df = np.array_split(df, 4)
    
    
    #iterate over blocks in excel file
    for block in split_df:
        '''
        get the columns for scanner offset, decision onsets, and feedback onsets 
        This uses the index of columns, which switches between the social and non-social files
        
        ''' 
        condition_type = 'social' if 'Social' in df.iloc[0][0] else 'doors'
        
        
        block_type = block['Procedure[Trial]'].iloc[0]
        
        block_type = 'positive' if ('Prize' in block_type) or ('Like' in block_type) else 'negative'
        
        
        
        if condition_type == 'social':
            offset_value = block.iloc[1, 56]
            decision_onsets = block.iloc[:, 82].dropna()  #get rid of nans  when possible
            
            
            
            
        
        else: #nonsocial
            offset_value = block.iloc[1, 40]
            decision_onsets = block.iloc[:, 51].dropna()
            
            
            
            
            
        #get decision onsets where R or L
        
        
        #print(decision_onsets, responses)
         
        
        all_cols = block.columns.tolist()
        for i in all_cols:
            if '.RESP' in i and ('Faces' in i or 'Doors' in i):
                response_column = i
            else:
                pass
        
        
        response_b = block.loc[block[response_column].eq('b')]
        response_g = block.loc[block[response_column].eq('g')]
        
        
    
        
        
        if condition_type == 'social':
            response_b_onsets = response_b.iloc[:, 82].dropna()
            response_g_onsets = response_g.iloc[:, 82].dropna()#get rid of nans  when possible
        else: #nonsocial
            response_b_onsets = response_b.iloc[:, 51].dropna()
            response_g_onsets = response_g.iloc[:, 51].dropna()
        
        #alternate,cleaner method for above
        #offset_value = block.filter(regex='ScannerTrigger*..OffsetTime', axis = 1).iloc[0].iloc[0]
        
        #get feedback onsets for correct and incorrect
        correct_onsets = block.loc[block['TrialType'] =='win', 'Outcome.OnsetTime']
        incorrect_onsets = block.loc[block['TrialType'] =='loss', 'Outcome.OnsetTime']
        
        #assign run based on column named "Block"
        run = block['Block'].iloc[0] 
        
        #assign a or b on column named "Block Number" 
        if block['BlockNumber'].iloc[0] == 1 :
            block_a_b = 'a'
        elif block['BlockNumber'].iloc[0] == 2:
            block_a_b = 'b' 
        else:
            print("Error")       
        
        
        
        #add fdmean values to log 
        
        
        
        
        
        
        #instruction type, as a string, is determined within block 
        instruction_type = block['Procedure[Trial]'].iloc[0][:-4] + 's.OnsetTime'
         
        #make scanner trigger adjustements
        
        adjusted_decision_onsets = decision_onsets.subtract(offset_value)
        
        adjusted_response_b_onsets = response_b_onsets.subtract(offset_value)
        adjusted_response_g_onsets = response_g_onsets.subtract(offset_value)
        
        adjusted_correct_onsets = correct_onsets.subtract(offset_value)
        adjusted_incorrect_onsets = incorrect_onsets.subtract(offset_value)
           
        #use first and last events to get experiment time 
        unadj_start = block[instruction_type].iloc[0]
        unadj_end = block['ITI.OffsetTime'].iloc[-1]
        
        block_start = (unadj_start - offset_value)
        block_end = (unadj_end - offset_value)
        dur = unadj_end - unadj_start
           
        #TR is 2.1 s
        trs = dur/2100
        
            
        '''
        B trials need to be given a new 'zero point'
        '''
        
        #print(block_a_b, adjusted_response_b_onsets)
        
        #reset b runs to block onsets, additional TR correction 
        if block_a_b == 'b':
            #print(True)
            #check for negatives
            #data correspond to which part of run
            
            #rename variables here, double check values
            
            adjusted_decision_onsets = adjusted_decision_onsets.subtract(block_start)
            
            adjusted_response_b_onsets = adjusted_response_b_onsets.subtract(block_start)
            adjusted_response_g_onsets = adjusted_response_g_onsets.subtract(block_start)
            
            adjusted_correct_onsets = adjusted_correct_onsets.subtract(block_start)
            adjusted_incorrect_onsets = adjusted_incorrect_onsets.subtract(block_start)
            
        else:
            pass
    
        #print(adjusted_response_b_onsets)
        
        instruct_list = [0., 5., 1.]
        
        #write EVs to file
        #event_writer('correct', adjusted_correct_onsets, 1., condition_type)
        #event_writer('incorrect', adjusted_incorrect_onsets, 1., condition_type)
        #event_writer('decision', adjusted_decision_onsets, 3., condition_type)
        
        #event_writer('Rdecision', adjusted_response_b_onsets, 3., block_a_b)
        #event_writer('Ldecision', adjusted_response_g_onsets, 3., block_a_b)
      
        #with open(f'../behavior/EVfiles/{sub}/{condition_type}/run-0{run}_{block_type}_instruction.txt', 'w', newline='') as csvfile:
        #        writer = csv.writer(csvfile, delimiter='\t',
        #                        quotechar='|', quoting=csv.QUOTE_MINIMAL)
        #        writer.writerow(instruct_list)

        #Add Block Info To Log
        #d1 = {sub + f'_{condition_type}_'+'_run_' + str(run) + '_block_' + block_a_b: [start, stop, block_length, block_a_b, condition_type, run, trs, block_type]}
        #interm.update(d1)

        #slicing the block out of the NIFTI  file
        
        if trs > 66:
            block_length = 'long'
            stop = 66
            if block_a_b == 'a':
                start = 0
            elif block_a_b == 'b' and block_length == 'long':
                start = 64
        elif trs < 65:
            block_length = 'short'
            stop = 64
            if block_a_b == 'a':
                start = 0
            elif block_a_b == 'b' and block_length == 'short':
                start = 66
    

        d1 = {sub + f'_{condition_type}_'+'_run_' + str(run) + '_block_' + block_a_b: [start, stop, block_length, block_a_b, condition_type, run, trs, block_type]}
        interm.update(d1)

        
        #if condition_type == 'social':
        #    infile = f'../bids/sub-{sub}/func/sub-{sub}_task-srSocial_run-0{run}_bold.nii.gz' 
        #    outfile = f'../bids/sub-{sub}/func/sub-{sub}_task-srSocial{block_a_b}_run-0{run}_bold.nii.gz'
        #    !fslroi $infile $outfile $start $stop 
        #else:
        #    infile = f'../bids/sub-{sub}/func/sub-{sub}_task-srDoors_run-0{run}_bold.nii.gz' 
        #    outfile = f'../bids/sub-{sub}/func/sub-{sub}_task-srDoors{block_a_b}_run-0{run}_bold.nii.gz'
        #    !fslroi $infile $outfile $start $stop 

In [87]:
#Write Log Out
#df = pd.DataFrame(interm).transpose()
#df.columns = ['Starting Volume', 'Length in Volumes', 'Long or Short', 'Block Type', 'Condition', 'Run Number', 'N Volumes', 'Instruction Valence']
#df.to_csv('test_output.csv')

# Level One Analysis

In [141]:
import glob
from csv import reader


for i in glob.glob('../behavior/EVfiles/*'):
    sub_number = i[20:24]
    print(sub_number)
    with open('test_output.csv', 'r') as read_obj:
        csv_reader = reader(read_obj)
        # Iterate over each row in the csv using reader object
        for row in csv_reader:
            if sub_number == row[0][:4]:
                trial_type = row[0][-1:]
                nvols = row[2]
                condition = row[0][5:11]
                condition = condition.replace("_", "").capitalize()
                print(trial_type, nvols, condition)
                #!./models/L1_task-sr$condition$trial_type\_\model-01.sh $sub_number 2 0 6 $nvols
                #!./models/L1_task-sr$condition$trial_type\_\model-01.sh $sub_number 1 0 6 $nvols
        read_obj.close()

# L1 Tests

In [91]:
test_file = pd.read_csv('/content/drive/My Drive/social_doors_college/code/test_output.csv')

print(test_file)

import glob
import pandas as pd
import re
import os
import nibabel as nib
import numpy as np

raw_data = glob.glob("../behavior/raw/s*")
EV_files = glob.glob('../behavior/EVfiles/*/*/*')
test_file = pd.read_csv('test_output.csv')
nii_root = '../bids/s*/func/*bold.nii.gz'



for file in test_file.iterrows():
	file_type = file[1][0]
	#get file parameters 
	run = file[1][6]
	block_letter = file_type[-1]
	sub = file_type[0:4]
	#expected_start = file[1][1]
	expected_length = file[1][2]
	condition = file[1][5]
	if condition == 'doors':
		soc_doors = f'Doors{block_letter}'
	elif condition == 'social':
		soc_doors = f'Social{block_letter}'
	else:
		print('error')
	
	trs = file[1][7]
	if trs > 65:
		slices = 66
	elif trs <= 65:
		slices = 64
	print(sub, run, block_letter, trs, slices == expected_length, slices, expected_length)
	#nii
	nii = f"sub-{sub}_task-sr{soc_doors}_run-0{run}_bold.nii.gz"
	nii_path = f'../bids/sub-{sub}/func/' + nii
	mri = nib.load(nii_path)
	print(mri.shape[3] == expected_length)

                     Unnamed: 0  Starting Volume  ...  N Volumes Instruction Valence
0     3847_doors__run_1_block_a                0  ...  66.333333            positive
1     3847_doors__run_1_block_b               66  ...  63.333333            negative
2     3847_doors__run_2_block_a                0  ...  66.333333            negative
3     3847_doors__run_2_block_b               66  ...  63.333333            positive
4    3847_social__run_1_block_a                0  ...  66.333333            positive
..                          ...              ...  ...        ...                 ...
235   3890_doors__run_2_block_b               66  ...  63.333333            positive
236  3890_social__run_1_block_a                0  ...  66.333333            positive
237  3890_social__run_1_block_b               66  ...  63.333333            negative
238  3890_social__run_2_block_a                0  ...  66.333333            negative
239  3890_social__run_2_block_b               66  ...  63.333333 

Example Model

# Level Two Analysis

In [65]:
import json
import pandas as pd
import numpy as np
from statistics import mean
import glob

In [ ]:
def extract_fd_vals(json_file):
    f = open(json_file)
    data = json.load(f)
    fd_data = data['fd_mean'] 
    f.close() 
    return fd_data

In [ ]:
def mean_fd_val(sub):
    fd_list = []
    for json_file in glob.glob(f'../derivatives/mriqc/derivatives/sub-{sub}*_bold.json'):
        fd_list.append(extract_fd_vals(json_file))
    return np.mean(fd_list)

def trial_fd_val(sub, run, condition, block):
    y = glob.glob(f'../derivatives/mriqc/derivatives/sub-{subnum}_task-sr{condition}{block}_run-0{run}_bold.json')
    #print(sub)
    return  extract_fd_vals(y[0])

In [ ]:
test_file_ordering = []
    
for row in test_file.iterrows():
    run = row[1][6]
    subnum = row[1][0][:4]
    condition = row[1][5]
    condition = condition.capitalize()
    valence = row[1][8]
    block = row[1][4]
    
    mean_fd_sub = mean_fd_val(subnum)
    
    mean_fd_run = trial_fd_val(subnum, run, condition, block)
    
    fd = (mean_fd_run - mean_fd_sub)
    
    test_file_ordering.append([subnum, f"L1_task-sr{condition}{block}_model-01_type-act_run-0{run}_sm-6_variant-dctAROMAnonaggr.feat", valence, fd])
            
    df = pd.DataFrame(data = test_file_ordering, columns = ['subject', 'L1', 'inst. valence', 'demeaned fd'])

    
    
per_sub = np.array_split(df, 30)

for i in per_sub:
    sub = i.iloc[0][0]
    
    
    sorted_vals = i.sort_values(by = 'inst. valence')
    
    l1_models = sorted_vals.iloc[0:, 1].tolist()
    l1_command = " ".join([j for j in l1_models])
    print(sub, l1_command)
    
    sorted_vals = i.sort_values(by = 'inst. valence')
    fd_list = sorted_vals.iloc[0:, 3].to_list()
    fd_command = " ".join([str(j) for j in fd_list])
    print(sub, len(l1_models) == 8)

    
    #!echo ./models/L2_combine_socialDoors.sh $sub  $l1_command $fd_command

# L2 Tests

# Group Level Mixed Effects

# Group Level Descriptive

In [88]:
print(Dataset.show())

<class '__main__.Dataset'>


# Plotting